![Learning Tree logo](images/logo.png)

# Hi-Lo refactored to modules

## Objective

Refactor the Hi-Lo game we created in a previous exercise to place the game engine in a separate module.

## Create a new app

Create a new app called `hi-lo_modules` and open it.

#### <font color="green">_Solution_</font>

## Review the existing code

The existing implementation is all contained within a single source file---mixing the game engine with the UI.

In [ ]:
use rand::Rng;
use std::io;

fn get_secret() -> u32 {
    rand::thread_rng().gen_range(1..101)
}

#[derive(PartialEq)]
enum GuessStatus {
    Uninitialized,
    TooLow,
    TooHigh,
    Exact,
}

struct HiLo {
    secret: u32,
    attempts: u32,
}

impl HiLo {
    fn new() -> HiLo {
        let hi_lo = HiLo {
            secret: get_secret(),
            attempts: 0,
        };

        hi_lo
    }

    fn make_guess(&mut self, guess: u32) -> GuessStatus {
        self.attempts += 1;

        if guess < self.secret {
            GuessStatus::TooLow
        } else if guess > self.secret {
            GuessStatus::TooHigh
        } else {
            GuessStatus::Exact
        }
    }
}

fn read_guess() -> u32 {
    println!("What number between 1 and 100 am I thinking of?");

    let mut s = String::new();

    io::stdin()
        .read_line(&mut s)
        .expect("Failed to read from input");

    s.trim().parse::<u32>().unwrap()
}

fn main() {
    let mut hi_lo = HiLo::new();

    let mut guess_status = GuessStatus::Uninitialized;

    while guess_status != GuessStatus::Exact {
        let guess = read_guess();

        guess_status = hi_lo.make_guess(guess);

        match guess_status {
            GuessStatus::TooLow => println!("Awww... {} is too low.", guess),
            GuessStatus::TooHigh => println!("Steady... {} is too high!", guess),
            _ => println!(
                "Congratulations! You took {} attempt(s) to guess I was thinking of {}.",
                hi_lo.attempts, hi_lo.secret
            ),
        }
    }
}

## Add dependencies

Add the `rand` (0.8.4) crate to the list of dependencies in `Cargo.toml`.

#### <font color="green">_Solution_</font>

## Create a game engine module

Create a new `game.rs` in the same location as the `main.rs` file. This will contain our game engine code.

Copy the following code to `game.rs`.

- `rand::Rng` use declaration
- `GuessStatus` enum
- `HiLo` struct (including the method implementations)

#### <font color="green">_Solution_</font>

In [ ]:
// game.rs

use rand::Rng;

#[derive(PartialEq)]
pub enum GuessStatus {
    Uninitialized,
    TooLow,
    TooHigh,
    Exact,
}

pub struct HiLo {
    pub secret: i32,
    pub attempts: i32,
    last_guess: Option<i32>,
}

impl HiLo {
    pub fn new() -> HiLo {
        let mut hi_lo = HiLo {
            secret: 0,
            attempts: 0,
            last_guess: None,
        };

        hi_lo.reset();

        hi_lo
    }

    pub fn reset(&mut self) {
        self.secret = rand::thread_rng().gen_range(1..101);
        self.attempts = 0;
        self.last_guess = None;
    }

    pub fn make_guess(&mut self, guess: i32) -> GuessStatus {
        self.attempts += 1;
        self.last_guess = Some(guess);

        if guess < self.secret {
            GuessStatus::TooLow
        } else if guess > self.secret {
            GuessStatus::TooHigh
        } else {
            GuessStatus::Exact
        }
    }
}

## Use the game engine module

The game engine module will be used by code in `main.rs` to implement the game. `main.rs` will contain all the UI code, meaning we could re-use `game.rs` in a web-app, for example.

Copy the definition of the `read_guess` function into `main.rs`---not forgetting the `std::io` use declaration it requires.

#### <font color="green">_Solution_</font>

In [ ]:
// Use declarations generally go at the top of the file
use std::io;

fn read_guess() -> i32 {
    println!("What number between 1 and 100 am I thinking of?");

    let mut s = String::new();

    io::stdin()
        .read_line(&mut s)
        .expect("Failed to read from input");

    s.trim().parse().unwrap()
}

Declare the `game` module in `main.rs`.

#### <font color="green">_Solution_</font>

In [ ]:
mod game;

Copy the `main` function defintion to `main.rs` and update all the reference to the `HiLo` struct to include its module. 

#### <font color="green">_Solution_</font>

In [ ]:
fn main() {
    let mut hi_lo = game::HiLo::new();

    let mut guess_status = game::GuessStatus::Uninitialized;

    while guess_status != game::GuessStatus::Exact {
        let guess = read_guess();

        guess_status = hi_lo.make_guess(guess);

        match guess_status {
            game::GuessStatus::TooLow => println!("Awww... {} is too low.", guess),
            game::GuessStatus::TooHigh => println!("Steady... {} is too high!", guess),
            _ => (),
        }
    }

    println!(
        "Congratulations! You took {} attempt(s) to guess I was thinking of {}.",
        hi_lo.attempts, hi_lo.secret
    );
}

`main.rs` function should now be as follows.

In [ ]:
// main.rs

use std::io;

mod game;

fn read_guess() -> i32 {
    println!("What number between 1 and 100 am I thinking of?");

    let mut s = String::new();

    io::stdin()
        .read_line(&mut s)
        .expect("Failed to read from input");

    s.trim().parse().unwrap()
}

fn main() {
    let mut hi_lo = game::HiLo::new();

    let mut guess_status = game::GuessStatus::Uninitialized;

    while guess_status != game::GuessStatus::Exact {
        let guess = read_guess();

        guess_status = hi_lo.make_guess(guess);

        match guess_status {
            game::GuessStatus::TooLow => println!("Awww... {} is too low.", guess),
            game::GuessStatus::TooHigh => println!("Steady... {} is too high!", guess),
            _ => (),
        }
    }

    println!(
        "Congratulations! You took {} attempt(s) to guess I was thinking of {}.",
        hi_lo.attempts, hi_lo.secret
    );
}


## Build and run the app

#### <font color="green">_Solution_</font>

## Congratulations

You have refactored the Hi-Lo game to place the game engine in a separate module.